In [1]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level, load
import comet_ml
import os
from deta import Deta
from dotenv import load_dotenv
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

load_dotenv()
set_log_level("ERROR")

deta = Deta(os.environ.get("DETA_PROJECT_KEY"))
comet_ml_api_key = os.environ.get("COMET_ML_API_KEY")
api = comet_ml.api.API(comet_ml_api_key)

In [2]:
# DAILY VISITS WITH WEATHER
api.download_registry_model("drdevinhopkins", "daily-visits-with-weather",
                            version="1.0.0", output_path="comet-ml-models", expand=True, stage=None)

loaded_model = load("comet-ml-models/daily-visits-with-weather.np")


COMET INFO: Downloading registry model 'daily-visits-with-weather', version '1.0.0', stage None from workspace 'drdevinhopkins'...
COMET INFO: Unzipping model to '/Users/devinhopkins/Dropbox/Code/hourly-report-neuralprophet/notebooks/comet-ml-models' ...
COMET INFO: done!


TypeError: _unpickle_timestamp() takes exactly 3 positional arguments (4 given)

In [6]:

df = pd.read_csv(
    'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/daily-visits.csv')
df.ds = pd.to_datetime(df.ds)
df = df.sort_values(by='ds')
df = df.reset_index(drop=True)

last_timestamp = df.loc[len(df)-1].ds
len(df)

3916

In [7]:

weather = pd.read_csv(
    'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/daily-weather.csv')
weather = weather.rename(columns={'datetime': 'ds'})
weather.ds = pd.to_datetime(weather.ds)
weather = weather[['ds', 'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipcover', 'snow',
                   'snowdepth', 'windgust', 'windspeed', 'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'moonphase']]
weather = weather.sort_values(by='ds', ascending=True)

weather.tail()

,ds,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,snowdepth,windgust,windspeed,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,moonphase
1130,2023-02-04,-12.2,-26.5,-18.2,-17.8,-34.0,-25.2,-20.7,80.6,3.9,...,53.9,29.5,13.7,1028.1,99.1,12.2,71.3,6.2,3,0.49
1131,2023-02-05,-9.9,-17.7,-12.7,-16.9,-26.3,-21.1,-15.5,79.4,3.8,...,63.0,51.5,27.7,1021.5,72.0,17.4,111.9,9.8,5,0.50
1132,2023-02-06,-8.5,-29.0,-16.6,-14.6,-34.1,-22.3,-19.2,80.2,0.4,...,61.7,42.1,15.5,1039.3,66.3,18.0,91.1,7.9,3,0.50
1133,2023-02-07,-3.0,-14.7,-7.8,-9.2,-20.9,-14.3,-9.8,85.5,0.2,...,59.8,61.6,27.7,1034.2,73.6,18.1,100.4,8.7,4,0.52
1134,2023-02-08,-0.3,-8.9,-5.0,-2.2,-12.8,-8.3,-5.7,95.1,19.8,...,62.5,31.0,13.0,1022.7,99.9,5.7,30.0,2.7,1,0.53


In [8]:

df = df.merge(weather, how='right', on='ds', suffixes='').


df

,ds,y,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,snowdepth,windgust,windspeed,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,moonphase
0,2020-01-01,NaN,1.1,-0.8,0.2,-0.6,-3.6,-1.9,-1.6,87.8,...,0.0,42.5,7.9,999.8,0.8,332.7,33.2,2.8,2,0.17
1,2020-01-02,NaN,2.6,-1.0,0.9,2.2,-2.5,-0.5,-2.4,78.7,...,0.0,31.7,7.7,1005.9,0.8,337.0,27.4,2.4,1,0.22
2,2020-01-03,NaN,5.8,2.8,4.8,5.0,2.6,4.0,0.7,74.9,...,0.0,NaN,7.6,1005.3,0.8,336.7,25.8,2.3,1,0.26
3,2020-01-04,NaN,3.2,-1.9,0.5,3.2,-5.8,-1.7,-2.9,78.8,...,0.0,NaN,11.2,1006.7,0.8,335.4,26.5,2.2,1,0.31
4,2020-01-05,NaN,-1.8,-9.9,-5.3,-4.0,-11.8,-8.1,-10.1,69.1,...,1.9,33.5,13.1,1009.7,0.7,338.1,53.4,4.6,3,0.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,2023-02-04,NaN,-12.2,-26.5,-18.2,-17.8,-34.0,-25.2,-20.7,80.6,...,53.9,29.5,13.7,1028.1,99.1,12.2,71.3,6.2,3,0.49
1131,2023-02-05,NaN,-9.9,-17.7,-12.7,-16.9,-26.3,-21.1,-15.5,79.4,...,63.0,51.5,27.7,1021.5,72.0,17.4,111.9,9.8,5,0.50
1132,2023-02-06,NaN,-8.5,-29.0,-16.6,-14.6,-34.1,-22.3,-19.2,80.2,...,61.7,42.1,15.5,1039.3,66.3,18.0,91.1,7.9,3,0.50
1133,2023-02-07,NaN,-3.0,-14.7,-7.8,-9.2,-20.9,-14.3,-9.8,85.5,...,59.8,61.6,27.7,1034.2,73.6,18.1,100.4,8.7,4,0.52


In [47]:


# future = pd.concat(
#     [df.tail(4*7+1), weather[weather.ds > last_timestamp].head(8)])
# future

In [48]:


forecast = loaded_model.predict(df)
forecast

Predicting: 0it [00:00, ?it/s]

,ds,y,yhat1,yhat2,yhat3,yhat4,yhat5,yhat6,yhat7,yhat8,...,future_regressor_snowdepth,future_regressor_solarenergy,future_regressor_solarradiation,future_regressor_temp,future_regressor_tempmax,future_regressor_tempmin,future_regressor_uvindex,future_regressor_visibility,future_regressor_windgust,future_regressor_windspeed
0,2020-01-01,208.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,332.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-03,332.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-04,253.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-05,253.0,204.468063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.354322,6.661678,-10.030825,3.464021,2.136695,0.929446,3.517778,1.459952,-3.864938,-3.382291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,233.754929,...,-10.126144,6.994762,-10.999467,2.698961,1.835044,0.673693,2.345185,0.017704,-4.144331,-7.906654
1128,2023-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-10.144794,7.494389,-11.838957,2.507696,1.759632,0.311895,2.345185,0.051817,-4.144331,-7.906654
1129,2023-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-9.921012,17.819990,-26.519262,-0.892570,-0.263945,-0.449128,5.862963,0.104066,-3.213021,-2.855181
1130,2023-02-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-10.051552,9.326349,-13.883867,0.722556,0.829540,-0.106044,3.517778,0.052681,-3.244065,-3.645846


In [49]:

forecast_trimmed = forecast[forecast.y.isnull()].set_index('ds')
forecast_trimmed.to_csv('forecasts/daily_visits_with_weather_raw.csv')
forecast_trimmed

,y,yhat1,yhat2,yhat3,yhat4,yhat5,yhat6,yhat7,yhat8,yhat9,...,future_regressor_snowdepth,future_regressor_solarenergy,future_regressor_solarradiation,future_regressor_temp,future_regressor_tempmax,future_regressor_tempmin,future_regressor_uvindex,future_regressor_visibility,future_regressor_windgust,future_regressor_windspeed
ds,,,,,,,,,,,,,,,,,,,,,
2023-01-25,NaN,229.191879,226.979996,219.871048,222.778244,226.000000,233.743942,227.924744,224.334732,224.211166,...,-5.296179,9.159807,-14.077596,2.826471,1.973301,0.792212,3.517778,0.051817,-4.470290,-8.389838
2023-01-26,NaN,NaN,227.520630,225.329102,220.521133,223.137482,226.995193,235.044479,228.099487,223.955490,...,-9.305611,4.996259,-7.533882,3.995312,2.212108,1.154010,2.345185,0.021591,-5.370557,-10.278650
2023-01-27,NaN,NaN,NaN,231.268921,229.132187,220.853165,223.994965,227.841064,236.904083,231.599670,...,-7.757783,18.153072,-27.186548,2.422689,1.470549,0.742309,4.690371,0.070385,-4.252984,-8.170210
2023-01-28,NaN,NaN,NaN,NaN,192.045639,190.570496,180.874146,184.056915,188.281647,195.908112,...,-9.231016,13.989524,-20.621309,3.867802,2.438347,1.054204,3.517778,0.090248,-7.605702,-14.056274
2023-01-29,NaN,NaN,NaN,NaN,NaN,188.802704,187.155594,178.317902,181.990234,185.257263,...,-9.529393,5.995511,-8.717777,2.337683,1.282017,0.810926,1.172593,0.015545,-3.135412,-6.501027
2023-01-30,NaN,NaN,NaN,NaN,NaN,NaN,258.436707,256.157990,247.615570,250.821213,...,-10.349927,11.491395,-16.918943,2.040159,1.143760,0.636265,2.345185,0.062181,-1.676359,-3.250514
2023-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240.667389,238.032135,231.116760,...,-10.238036,12.990273,-18.985380,1.912649,1.080917,0.623789,3.517778,0.052681,-1.521140,-3.162662
2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,233.754929,233.030899,...,-10.126144,6.994762,-10.999467,2.698961,1.835044,0.673693,2.345185,0.017704,-4.144331,-7.906654
2023-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232.627945,...,-10.144794,7.494389,-11.838957,2.507696,1.759632,0.311895,2.345185,0.051817,-4.144331,-7.906654


In [50]:

start = forecast_trimmed.index.tolist()[0]
forecast_length = len(forecast_trimmed.index.tolist())
i = 0
timestamp = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output_list = []
for ds in pd.date_range(start=start, periods=forecast_length, freq='D'):
    i = i+1
    forecast_output_list.append(
        {'ds': ds, 'visits': forecast_trimmed.loc[ds]['yhat'+str(i)], 'timestamp': timestamp})
forecast_output = pd.DataFrame(forecast_output_list)
forecast_output.to_csv('forecasts/daily_visits_with_weather.csv', index=False)
forecast_output

,ds,visits,timestamp
0,2023-01-25,229.191879,2023-01-25 10:39:09
1,2023-01-26,227.520630,2023-01-25 10:39:09
2,2023-01-27,231.268921,2023-01-25 10:39:09
3,2023-01-28,192.045639,2023-01-25 10:39:09
4,2023-01-29,188.802704,2023-01-25 10:39:09
5,2023-01-30,258.436707,2023-01-25 10:39:09
6,2023-01-31,240.667389,2023-01-25 10:39:09
7,2023-02-01,233.754929,2023-01-25 10:39:09
8,2023-02-02,232.627945,2023-01-25 10:39:09
9,2023-02-03,225.462463,2023-01-25 10:39:09
